In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from itertools import tee

In [2]:
df = pd.read_csv("Clusters+vert_cercanos.csv", delimiter=",")

In [3]:
df_seguridad_policia = pd.read_csv("seguridad_policia_oficiales.csv", delimiter=";")

In [5]:
df_seguridad_policia

,Id,Latitud_seguridad_policia,Longitud_seguridad_policia
0,1,4.167179,-73.623657
1,2,4.123192,-73.577690
2,3,4.148795,-73.634969
3,4,4.077741,-73.670454
4,5,4.058403,-73.672695
5,6,4.156303,-73.631032
6,7,4.121043,-73.642440
7,8,4.122414,-73.632141
8,9,4.165158,-73.641056
9,10,4.136793,-73.645865


In [7]:
array_id=df_seguridad_policia.Id.values

In [8]:
df2=df.copy(deep=True)

In [9]:
for col_name in array_id:
    df2[col_name] = None

In [2]:
#Se quita por motivos de seguridad
key_API_google=""

In [34]:
#aca origenes son destinos y destinos son origenes por que estamos midiendo cuanto se demora la policia en llegar allá

In [20]:
origenes = [row['Latitud_seguridad_policia'].astype(str)+"%2C"+row['Longitud_seguridad_policia'].astype(str) for index, row in df_seguridad_policia.iterrows()]

In [21]:
destinos = [row['LATITUD'].astype(str)+"%2C"+row['LONGITUD'].astype(str) for index, row in df.iterrows()]

In [22]:
todas_solicitudes=[]
for i in range(len(origenes)):
    solicitudes=[]
    for j in range(len(destinos)):
        solicitud="https://maps.googleapis.com/maps/api/distancematrix/json?destinations="+destinos[j]+"&origins="+origenes[i]+"&key="+key_API_google
        solicitudes.append(solicitud)
    todas_solicitudes.append(solicitudes)

In [23]:
todas_solicitudes2 = pd.DataFrame(todas_solicitudes)

In [18]:
def solicitud_maps(url):
    try:
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        x = response.json()
        drive_time_seconds = x['rows'][0]['elements'][0]['duration']['value']
        distance_meters = x['rows'][0]['elements'][0]['distance']['value']
        return [drive_time_seconds, distance_meters]
    except KeyError:
        return 0

In [26]:
transposed_todas_solicitudes2 = todas_solicitudes2.transpose()

In [28]:
df2[1] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[0]), axis=1)


In [30]:
df2[2] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[1]), axis=1)
df2[3] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[2]), axis=1)
df2[4] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[3]), axis=1)
df2[5] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[4]), axis=1)
df2[6] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[5]), axis=1)
df2[7] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[6]), axis=1)
df2[8] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[7]), axis=1)
df2[9] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[8]), axis=1)
df2[10] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[9]), axis=1)
df2[11] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[10]), axis=1)
df2[12] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[11]), axis=1)
df2[13] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[12]), axis=1)

In [32]:
df3=df2.copy(deep=True)
df3 = df3.drop(array_id, axis=1)
for col in array_id:
    df3[col] = df2[col].apply(lambda x: x[0])

df3.to_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SEGURIDAD_POLICIA.csv",index=True)

In [33]:
df4=df2.copy(deep=True)
df4 = df4.drop(array_id, axis=1)
for col in array_id:
    df4[col] = df2[col].apply(lambda x: x[1])
    
df4.to_csv("DISTANCIAS_METROS_CLUSTERS_SEGURIDAD_POLICIA.csv",index=True)